In [ ]:
import os
import importlib  # used to reload modules if necessary
path_to_frontend = '..'
os.chdir(path_to_frontend)
print("current working directory: " + os.getcwd())
import q100viz.session as session
import q100viz.devtools as devtools
import q100viz.graphics.graphs as graphs

## Simulation

In [2]:
# prepare reference data: select buildings without decisions
session.buildings.df['selected'] = True
session.buildings.df['group'] = 1

In [ ]:
# simulate until 2045
session.simulation.setup(input_max_year=2045, export_graphs=False)
session.simulation.run()

## Export graphs:

In [ ]:
############# neighborhood data #############
# neighborhood total emissions:
graphs.export_individual_graph(
    csv_name="/emissions/CO2_emissions_neighborhood.csv",
    data_folders=session.simulation.output_folders,
    columns=['emissions_neighborhood_accu'],
    title_="jährlich kumulierte Gesamtemissionen des Quartiers",
    outfile=session.simulation.current_output_folder + "/emissions/CO2_emissions_neighborhood.png",
    xlabel_="Jahr",
    ylabel_="$CO_{2}$-Äquivalente (t)",
    x_='current_date',
    convert_grams_to_tons=True,
    compare_data_folder=session.simulation.current_output_folder + "/../../precomputed/simulation_defaults"
)

# neighborhood total energy prices prognosis:
graphs.export_neighborhood_total_data(
    csv_name="/energy_prices/energy_prices_total.csv",
    data_folders=[session.simulation.current_output_folder],
    columns=['gas_price', 'power_price', 'oil_price'],
    labels_=['Gaspreis', 'Strompreis', 'Ölpreis',],
    title_="generelle Energiepreise nach Energieträger",
    outfile=session.simulation.current_output_folder + "/energy_prices/energy_prices_total.png",
    xlabel_="Jahr",
    ylabel_="Preis (ct/kWh)",
    x_='current_date',
    label_show_iteration_round=False
    # compare_data_folder=session.simulation.current_output_folder + "/../../precomputed/simulation_defaults"
)

########## individual buildings data ########
for group_df in session.buildings.list_from_groups():
    if group_df is not None:
        for idx in group_df.index:

            # export emissions graph:
            graphs.export_default_graph(
                csv_name="/emissions/CO2_emissions_{0}.csv".format(
                    idx),
                data_folders=session.simulation.output_folders,
                csv_columns=['building_household_emissions'],
                title_="ø-CO2-Emissionen (monatlich berechnet)",
                # outfile=session.simulation.current_output_folder +
                # "/emissions/CO2_emissions_{0}.png".format(idx),
                xlabel_="Jahr",
                ylabel_="$CO_{2}$-Äquivalente (kg)",
                x_='current_date',
                convert_grams_to_kg=True,
                figsize=(16,12),  # inches
                show_legend=False
            )

            # export energy prices graph:
            graphs.export_individual_graph(
                csv_name="/energy_prices/energy_prices_{0}.csv".format(
                    idx),
                data_folders=session.simulation.output_folders,
                columns=['building_household_expenses_heat',
                        'building_household_expenses_power'],
                labels_=['Wärmekosten', 'Stromkosten'],
                outfile=session.simulation.current_output_folder +
                "/energy_prices/energy_prices_{0}.png".format(idx),
                title_="Energiekosten",
                xlabel_="Jahr",
                ylabel_="€/Monat",
                x_='current_date',
                figtext=
                    str(idx) + " "
                    + str(group_df.loc[idx, 'address']) + " "
                    + str(group_df.loc[idx, 'type'])
                    + "\nø-spez. Wärmeverbrauch: "
                    + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
                    + ", ø-spez. Stromverbrauch: "
                    + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
                    if session.VERBOSE_MODE else "",
                figsize=(16,12),  # inches
                overwrite_color='lightgray',
                label_show_iteration_round=False
            )

## Copy data

after the simulation is done, copy the freshly created folder to `..data/precomputed/simulation_defaults`

In [ ]:
import shutil

source = session.simulation.current_output_folder
destination = session.simulation.reference_data_folder + "_ylim0"

shutil.copytree(source, destination, dirs_exist_ok=True)

---
## test stuff:

In [ ]:

# create individual emissions graphs from precomputed reference data:
for idx in session.buildings.df.index:
    graphs.export_default_graph(
        csv_name="/emissions/CO2_emissions_{0}.csv".format(
            idx),
        data_folders=[session.simulation.reference_data_folder],
        csv_columns=['building_household_emissions'],
        title_="CO2-Emissionen (monatlich berechnet)",
        outfile=session.simulation.reference_data_folder +
        "/emissions/CO2_emissions_{0}.png".format(idx),
        xlabel_="Jahr",
        ylabel_="ø-Emissionen [$kg_{CO2,eq}$]",
        x_='current_date',
        convert_grams_to_kg=True,
        # compare_data_folder=session.simulation.current_output_folder + "/../../precomputed/simulation_defaults",
        figsize=(16,12)  # inches
    )


In [ ]:
# example graph:
importlib.reload(graphs)
idx = "1.02"

# export emissions graph:
graphs.export_default_graph(
    csv_name="/emissions/CO2_emissions_{0}.csv".format(
        idx),
    data_folders=session.simulation.output_folders,
    csv_columns=['building_household_emissions'],
    title_="ø-CO2-Emissionen (monatlich berechnet)",
    # outfile=session.simulation.current_output_folder +
    # "/emissions/CO2_emissions_{0}.png".format(idx),
    xlabel_="Jahr",
    ylabel_="$CO_{2}$-Äquivalente (kg)",
    x_='current_date',
    convert_grams_to_kg=True,
    figsize=(16,12),  # inches
    show_legend=False
)

# export energy prices graph:
graphs.export_individual_graph(
    csv_name="/energy_prices/energy_prices_{0}.csv".format(
        idx),
    data_folders=session.simulation.output_folders,
    columns=['building_household_expenses_heat',
            'building_household_expenses_power'],
    labels_=['Wärmekosten', 'Stromkosten'],
    outfile=session.simulation.current_output_folder +
    "/energy_prices/energy_prices_{0}.png".format(idx),
    title_="Energiekosten",
    xlabel_="Jahr",
    ylabel_="€/Monat",
    x_='current_date',
    figtext=
        str(idx) + " "
        + str(group_df.loc[idx, 'address']) + " "
        + str(group_df.loc[idx, 'type'])
        + "\nø-spez. Wärmeverbrauch: "
        + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
        + ", ø-spez. Stromverbrauch: "
        + str(group_df.loc[idx, 'avg_spec_heat_consumption'])
        if session.VERBOSE_MODE else "",
    figsize=(16,12),  # inches
    overwrite_color='lightgray',
    label_show_iteration_round=False
)
